In [1]:
from Bio import Entrez, SeqIO
Entrez.email = 'curiousgeorge@ufl.edu'

#prompt user for organism and number of results desired
query= input('Enter organism name')
usermax=input('Enter maximum number of results ')

#send query to Entrez nucleotide database
handle = Entrez.esearch(db='nucleotide', term = query, field='organism', retmax=usermax)
record = Entrez.read(handle)
handle.close()

#get the 'IDList' field from the results
ids=(record["IdList"])
print(ids) #optional

#function to get sequences from the genbank IDs returned from the search and write to 'query_fastas.txt'
def get_sequences():
    open('query_fastas.txt','w')
    for seq_id in ids:
        handle = Entrez.efetch(db="nucleotide", id=seq_id, rettype="fasta", retmode="text",retmax=1)
        local_file=open('query_fastas.txt','a')  
        local_file.write(handle.read())
get_sequences()
handle.close()

Enter organism name tuber
Enter maximum number of results  20


['1843798390', '1843798389', '1837918107', '1837918106', '1837893017', '1837893016', '1634554516', '1634551058', '1634551057', '1634551056', '1835707986', '1835707985', '1835707984', '1835707983', '1835707982', '1835707981', '1835707980', '1786710977', '1786710975', '1786520428']
